In [1]:
"""
OBJETIVO : Definir y entrenar lso modelos. Ademas, recolectar los resultados para analizarlos mas adelante  
"""
Autor='Diego Paredes'

In [2]:
#Manejo de Datos
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import boxcox


#Machine learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import Callback

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix

#Librerias estandar (Extras)
import re
import os
import time
import random
from datetime import datetime

In [3]:
"""
DEFINIMOS EL PATH DEL PROYECTO 
"""
with open('../../path_base.txt') as f:
    path_base = f.read()
path_base

'C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/NewTesis'

In [4]:
"""
Variables generales
"""
path_imagenes = 'F:/GOES/'     

products = ['C13','C07','C08']
times   = ['10','20','30','40','50','00']


In [5]:
!python --version
print(tf. __version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

#Limitamos el GPU, en caso se necesite
gpus = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_virtual_device_configuration(gpus[0],
                                                        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])

Python 3.9.11
2.8.0
Num GPUs Available:  1


In [6]:
"""
Metodos para realizar el entrenamient - evaluacion del modelo
"""

'\nMetodos para realizar el entrenamient - evaluacion del modelo\n'

In [7]:
def getConv2D(p,run):    
    model = keras.Sequential()
    shape = (p['margen'][run],p['margen'][run],p['canales'][run])
    # Imagen    
   
    model.add(tf.keras.layers.Rescaling(1./65536))
    
    model.add(tf.keras.layers.Conv2D(128, (3,3), input_shape=(p['margen'][run],p['margen'][run],p['canales'][run]),padding='same', activation='relu'))    
    model.add(tf.keras.layers.MaxPooling2D())
    model.add(tf.keras.layers.Dropout(0.2))
    
    model.add(tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D())
    model.add(tf.keras.layers.Dropout(0.2))
    """
    model.add(tf.keras.layers.Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D())
    model.add(tf.keras.layers.Dropout(0.2))
    
    model.add(tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(tf.keras.layers.Dropout(0.2))
    
    model.add(tf.keras.layers.Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(tf.keras.layers.Dropout(0.1))
    """
    model.add(tf.keras.layers.GlobalMaxPool2D())
    
    return model    

In [8]:
def crearModelo(p,run):    
    convnet = getConv2D(p,run)
    shape = (p['tiempos'][run],p['margen'][run],p['margen'][run],p['canales'][run])
    
    input_1 = tf.keras.layers.Input(shape=shape)
    
    timeD = tf.keras.layers.TimeDistributed(convnet)(input_1)
    lstm = tf.keras.layers.LSTM(128)(timeD)
    
    
    listConcat = [lstm]
    listInputs = [input_1]
    
    if len(p['inputs'])>1:
        #Agregamos los otros atrbutos        
        for attr in p['inputs'][1:]:
            # The other input
            input_x = tf.keras.layers.Input(shape=(1,))
            listConcat.append(input_x)
            listInputs.append(input_x)

            
        # Concatenate
        final = tf.keras.layers.Concatenate()(listConcat) 
    
    dense_1 = tf.keras.layers.Dense(units=64, activation=tf.keras.activations.relu)(final)
    dense_2 = tf.keras.layers.Dense(units=32, activation=tf.keras.activations.relu)(dense_1)
    dense_3 = tf.keras.layers.Dense(units=32, activation=tf.keras.activations.relu)(dense_2)
    dense_4 = tf.keras.layers.Dense(units=32, activation=tf.keras.activations.relu)(dense_3)
    
    dense_F = tf.keras.layers.Dense(units=16, activation=tf.keras.activations.relu)(dense_4)
    
    # output
    if p['redTipo'] == 'Regresion':
        output = tf.keras.layers.Dense(units=1)(dense_F)      
    elif p['redTipo'] == 'Clasificacion':
        output = tf.keras.layers.Dense(units=1,activation=tf.keras.activations.sigmoid)(dense_F)
    else:
        print(f"No se pudo crear el modelo outputs no esta bien definido {p['redTipo']}")
        return -1      
    
    full_model = tf.keras.Model(inputs=listInputs, outputs=[output])

    return full_model

In [9]:
def getMetrics(modelType, lr, paciencia):
    
    if modelType == 'Clasificacion':    
        optimizer = keras.optimizers.Adam(learning_rate=lr) 
        
        #BinaryCrossentropy() #CategoricalCrossentropy()      
        loss_fn= keras.losses.BinaryCrossentropy()
        train_acc_metric = keras.metrics.BinaryCrossentropy()
        val_acc_metric = keras.metrics.BinaryCrossentropy()
        if paciencia:
            early_stopping = keras.callbacks.EarlyStopping(monitor="val_acc", patience=paciencia, mode="max")  
 
        
        metrics = ['acc', keras.metrics.TruePositives(),
                         keras.metrics.TrueNegatives(),
                         keras.metrics.FalsePositives(),
                         keras.metrics.FalseNegatives()]
        

    elif modelType == 'Regresion':
        optimizer = keras.optimizers.Adam(learning_rate=lr)
        loss_fn=keras.losses.MeanSquaredError()
        train_acc_metric = keras.metrics.MeanSquaredError()
        val_acc_metric = keras.metrics.MeanSquaredError()
        if paciencia:
            early_stopping = keras.callbacks.EarlyStopping(monitor="val_mse", patience=paciencia, mode="max")                                            
        metrics = ['mse']
        
        
    else:
        print('No se pudo crear las metricas')
        return -1    
         
        
    logs = Callback()
    callbacks = [logs]                     
    if paciencia:
        callbacks.append(early_stopping)
        
    metrics = {'optimizer': optimizer, 'loss_fn':loss_fn,'train_acc_metric': train_acc_metric,
               'val_acc_metric': val_acc_metric, 'metrics': metrics,'callbacks': callbacks}
    
    return metrics
        

In [10]:
def applyDA(img, DA):
    if DA == 1:
        return tf.image.flip_left_right(img)
    elif DA == 2:
        return tf.image.flip_up_down(img)
    elif DA == 3:
        img = tf.image.flip_left_right(img)
        return tf.image.flip_up_down(img)   
    else:
        return img

In [11]:
#Transformamos un filename tensor en una imagen
def read_png_file(item, value, p,run, path_base, products, times, DA=0):
    # imagenData[0] = XO     # imagenData[1] = XA     # imagenData[2] = Fecha
    imagenData = tf.strings.split(item['imagen'], sep='--')
    size = int(p['margen'][run] / 2)

    timeJoin = []
    for j in range(p['tiempos'][run]-1,-1,-1):
        filename = path_base + 'PNG/' + imagenData[2] + '/' + imagenData[2] + '_' + str(j) + '.png'        
        image_string = tf.io.read_file(filename)
        img_decoded = tf.io.decode_png(image_string, dtype=tf.uint16, channels=3)       
        
        if DA:
            img_decoded = applyDA(img_decoded, item['DA'])
                
        timeJoin.insert(0,img_decoded[int(imagenData[1]) - size:int(imagenData[1]) + size,
                                      int(imagenData[0]) - size:int(imagenData[0]) + size,
                                      0:p['canales'][run]])
 
        
    if p['tiempos'][run]==1:
        imagenData = tf.reshape(timeJoin[0],(p['margen'][run],p['margen'][run],p['canales'][run]))
    else:
        if p['meanMatrizImagen']:        
            img = tf.reduce_mean( timeJoin , axis=0 )
            imagenData = tf.reshape(img,(p['margen'][run],p['margen'][run],p['canales'][run]))
        else:
            img = tf.stack(timeJoin, axis=0)
            imagenData = tf.reshape(img,(p['tiempos'][run],p['margen'][run],p['margen'][run],p['canales'][run]))
        
    
    
    if len(p['inputs']) == 1:
        return imagenData, int(value)
    
    item['imagen'] = imagenData
    itemL = []
    for inpL in p['inputs']:
        itemL.append(item[inpL])
    
    if p['redTipo']=='Regresion':
        print('es regresion')
        return tuple(itemL), float(value)
    else:
        return tuple(itemL), int(value)


In [12]:
def splitDataset(p, run, dataset, path_imagenes, products, times,val_split= 0.2):
            
    if p['dsVal']:
        test = pd.read_csv(p['dsVal'])
        if p['dataset']:
            train =  dataset.sample(frac=p['dataset'])
            test = test.sample(frac=p['dataset'])
        else:
            train =  dataset
            
    else:
        # Escojemos una fraccion del dataset si se ha indicado           
        if p['dataset']:
            train, test = train_test_split(dataset.sample(frac=p['dataset']), test_size=val_split, shuffle=True)
        else:
            train, test = train_test_split(dataset, test_size=val_split, shuffle=True)
               
    inputsList = {}
    inputsListTest = {}
    # Agregamos un atributo para indicar que el dato va realizar DA
    if p['DA']:        
        inputsList['DA'] = train['DA'].tolist() 
        #inputsListTest['DA'] = test['DA'].tolist() 
        
        
    print(f'Tamaño del dataset: Train {len(train)}  - Val {len(test)}')    
    
    for inp in p['inputs']:
        inputsList[inp] = train[inp].tolist()  
        inputsListTest[inp] = test[inp].tolist()  
        
       
        
    train_dataset = tf.data.Dataset.from_tensor_slices(((inputsList),train[p['outputs']].tolist()))           
    val_dataset = tf.data.Dataset.from_tensor_slices(((inputsListTest),test[p['outputs']].tolist()))     
    
    train_dataset = train_dataset.map(lambda x ,y : read_png_file(x,y,p,run,path_imagenes,products,times,p['DA']))
    val_dataset = val_dataset.map(lambda x ,y : read_png_file(x,y,p,run,path_imagenes,products,times,False))#p['DA']
       
    
    train_dataset = train_dataset.batch(p['batch'])#.cache().prefetch(tf.data.AUTOTUNE)
    val_dataset = val_dataset.batch(p['batch'])#.prefetch(tf.data.AUTOTUNE)  
    
    
    
    return train_dataset, val_dataset

In [13]:
def crearDir(path, newDir):
    try:
        pathT = os.path.join(path, newDir)
        os.mkdir(pathT)
        return pathT
    except FileExistsError:
        return pathT
        pass
    except:
        print(f"No se pudo crear el directorio: {newDir}")
        pritn(f'Path base: {path}')
        pritn(f'Nuevo    : {newDir}')        
        return None

In [14]:
def inicializarVariables(repDir, params,ds):
    # Leemos dataset
    try:
        dataset = pd.read_csv(ds)        
        dsName = ds.split('/')[-1][:-4]
    except:
        print(f'No se pudo leer el dataset {ds}')
        return None
    
    statsDir = crearDir(repDir, dsName)
    excelFile = f'{statsDir}/Stats_{params["redTipo"]}_{params["outputs"]}_{datetime.today().strftime("%Y%m")}.xlsx'
    
    if params['record'] and not os.path.exists(excelFile):
        writer = pd.ExcelWriter(excelFile, engine = 'xlsxwriter')
        keys_values = params.items()
        strParams = {str(key): str(value) for key, value in keys_values}
        strParams['dsName'] = dsName
        pd.DataFrame(strParams,index=[0]).to_excel(writer, sheet_name = 'Informacion')          
        writer.save()
        
        
    return statsDir, excelFile, dataset            

In [15]:
def getCM(logs):
    lKeys = list(logs.keys())
    
    try:
        TN = int(logs[[x for x in lKeys if 'val_true_negatives' in x][0]])
        TP = int(logs[[x for x in lKeys if 'val_true_positives' in x][0]])
        FN = int(logs[[x for x in lKeys if 'val_false_negatives' in x][0]])
        FP = int(logs[[x for x in lKeys if 'val_false_positives' in x][0]])
    except:
        print(f'\nNo se pudo leer keys para la matriz de confucion en logs : {lKeys}')
        print(f'Se intento leer: val_true_negatives,val_true_positives, val_false_negatives y val_false_positives')
    
       
    y_true =  [0]*TN + [1]*TP + [1]*FN + [0]*FP
    _y_pred = [0]*TN + [1]*TP + [0]*FN + [1]*FP
    
    return TN, FP, FN, TP, np.array(y_true), np.array(_y_pred)

In [16]:
class CustomCB(Callback):
    """ Custom callback to compute metrics at the end of each training epoch"""
    def __init__(self, val_ds=None, WANDB=True):     
        self.val_ds = val_ds  
        self.history = {}
        self.wandb = WANDB
   
   
    def on_epoch_end(self, epoch, logs={}):  
        TN, FP, FN, TP, y_true, _y_pred = getCM(logs)
        
        self.history.setdefault('loss', []).append(logs['loss']) 
        self.history.setdefault('acc', []).append(logs['acc'])  
        self.history.setdefault('val_loss', []).append(logs['val_loss']) 
        self.history.setdefault('val_acc', []).append(logs['val_acc']) 
        
        
        self.history.setdefault('val_TN', []).append(TN) 
        self.history.setdefault('val_FP', []).append(FP)
        self.history.setdefault('val_FN', []).append(FN) 
        self.history.setdefault('val_TP', []).append(TP) 
        
            
        if self.wandb:
            wandb.log({"conf_mat" : wandb.plot.confusion_matrix(probs=None,
                                    preds=_y_pred, y_true=y_true,
                                    class_names=[0,1]),                   
                       'val_TN' :TN,'val_FN' :FN,'val_TP' :TP,'val_FP' :FP,
                       'val_acc': logs['val_acc'],'loss' : logs['loss'],
                       'val_loss': logs['val_loss'],'acc' : logs['acc']                  
                      })


In [17]:
def crearCallbacks(statsDir,dsName, params,run, metricas):
    CB = metricas['callbacks']
    
    idModel = datetime.today().strftime("%Y%m%d_%H%M%S")
    checkpoint_path = statsDir + '/Model_{epoch:02d}_' + f'{params["redTipo"]}_{params["outputs"]}_{idModel}.hdf5' 
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,                                                     
                                                     verbose=1)
    
    # Iniciamos WANDB
    if params['record']:        
        CB.append(cp_callback)   
     
    if params['WANDB']:
        config = dict(learning_rate=params['lr'], epochs = params['epocas'],
             batch_size =params['batch'],architecture="CNN", 
             num_classes = params['num_class'],)
        wandb.init(project=f'{params["Proyect"]}-({params["redTipo"]}-{params["outputs"]}-{len(params["inputs"])})',            
                   config=config,
                   name= f'Ex_{dsName}_({params["canales"][run]}-{params["tiempos"][run]}-{params["margen"][run]})_{idModel}')   
                                               
    return CB, idModel

In [18]:
def trainModel(params,path_imagenes, path_base ,products, times):       
        
    if '.csv' in params["dsDir"]:
        ds_files = [params["dsDir"]]
    else:
        ds_files = [f'{params["dsDir"]}{e}' for e in os.listdir(params["dsDir"])]
        ds_files = [e for e in ds_files if '.csv' in e]
        
    maxDS = len(ds_files) if params['maxDS']==-1 else params['maxDS']
    print(f'Cantidad de datasets encontrados : {len(ds_files)}')    
    print(f'Datasets a usar : {maxDS}')    
    
    all_result = []      
    
    repDir = crearDir(f'{path_base}/Archivos/Resultados', params["redTipo"])   
    repDir = crearDir(repDir, params["Proyect"])
    
    
    print(f'DIRECTORIO BASE : {repDir}')
    
    # Una iteracion por cada dataset existente
    ds_i = 0
    
    
    for ds in ds_files[ds_i:maxDS+1]:
        ds_i += 1       
        
        # Crear carpeta para DS y archivo de stats y el ds
        statsDir , statsFile , dataset = inicializarVariables(repDir, params,ds)         
            
        resultados = []    
        for run in range(params['runs']): 
            print('__________________________________________________')
            print(f'DATASET : {ds_i}/{maxDS}')            
            print('__________________________________________________')
            print(f'Inicio de la prueba N°: {(run+1)}/{params["runs"]}') 
            print(f'Dataset: {ds.split(".")[0]}')
            print(f'- Batch size:  {params["batch"]}')
            print('__________________________________________________')
            
           
            # Modelo 
            model = crearModelo(params,run) 
            metricas = getMetrics(params['redTipo'], params['lr'], params['paciencia'])
            model.compile(optimizer=metricas['optimizer'],loss=metricas['loss_fn'],metrics=metricas['metrics'],)
            
            # Dataset        
            train_dataset, val_dataset = splitDataset(params,run, dataset, path_imagenes, products, times, params['val_split'])
       
            # Creamos los callbacks
            CB, idModel = crearCallbacks(statsDir,ds.split('/')[-1][:-4], params,run, metricas)           

            if params['redTipo'] == 'Clasificacion':
                hist =  CustomCB(val_dataset, params['WANDB'])
                #CB.append(hist)
        
            
            #Entrenamos
            history = model.fit(train_dataset,batch_size=params['batch'],                            
                                epochs=params['epocas'],callbacks=CB,
                                validation_data=val_dataset,
                                validation_batch_size=params['batch'],
                                verbose=1)
            if params['redTipo'] != 'Clasificacion': 
                HistTemp = history
            else:
                HistTemp = history#hist
                
            resultados.append(HistTemp.history)
            
            # Guardamos las estadisticas
            if params['record']:                
                with pd.ExcelWriter(statsFile, mode="a", engine="openpyxl", if_sheet_exists='overlay') as writer:                    
                    tempDF = pd.DataFrame(HistTemp.history)
                    if params['redTipo'] == 'Clasificacion':
                        tempDF.columns = ['loss', 'acc', 'TP', 'TN', 'FP','FN','val_loss','val_acc','val_TP','val_TN','val_FP','val_FN']
                    tempDF.to_excel(writer,startrow=0,
                                    sheet_name=f'{run}-({params["canales"][run]}-{params["tiempos"][run]}-{params["margen"][run]})-{idModel}')
            if params['WANDB']:
                wandb.finish()
        
        all_result.append(resultados)
        
    return all_result

In [19]:
import wandb
from wandb.keras import WandbCallback
wandb.login()

wandb: Currently logged in as: diegoparedes (tesis2). Use `wandb login --relogin` to force relogin


True

In [20]:
"""
# Definimos las varibles para las iteraciones
Los parametros que van a cambiar son:
- Canales (products)
- Tiempos (Minutos de las imagenes)
- margen
"""

modelTipo = 'Clasificacion'
idProject = datetime.today().strftime("%Y%m%d_%H")
p_train = {
            # Reportes
          'directory': 'RNN'
          'Proyect'  : f'{modelTipo}_RNN_UMB1-DATO-ALT-2CONV-{idProject}', # TesisDiego
          'record'   : True,  # Grabar los resultados en  excels    
          'WANDB'    : False, # Grabar los resultados en WANDB
    
            # Datos del modelo
          'redTipo'  : modelTipo, # Clasificacion / Regresion
          'rnn'      : True,  # Redes recurrentes
          'inputs'   : ['imagen','umb1','dato','altura'], 
          'outputs'  : 'clase',       # clase / dato
          'num_class': 2,             # Solo se usa en el modelo de clasificacion
          'meanMatrizImagen' : False, # !!! RNN modelos SIEMPRE EN FALSE  !!!
          
        
            # Variables del entrenamiento          
          'lr'       : 0.001,
          'batch'    : 32,        
          'val_split': 1,     # ¡¡ Si dsVal existe, este valor se ignora !!
          'epocas'   : 40,  
          'paciencia': 10,        # 0 = No paciencia  
    
    
           # Dataset
          'dsDir'    : f'{path_base}/Archivos/Dataset/{modelTipo}/Entrenamiento/SplitConDA_20C_DM_V2/',  #SplitConDA_DM_DFAD_DUD
          'dsVal'    : f'{path_base}/Archivos/Dataset/{modelTipo}/Validacion/ClaseV2_DUD_ValidacionDS.csv',
          'maxDS'    : -1,      #-1 = Todos
          'dataset'  : 1,     # 1 = 100% del ds
          'DA'       : True,  # Usaulmente para clasificacion
                    
           # Hiper parametros 
          'canales'  : [3],
          'tiempos'  : [6],
          'margen'   : [30],
          'runs'     : 1
         }

In [21]:
# Forma del DS
ds_files = [f'{p_train["dsDir"]}{e}' for e in os.listdir(p_train["dsDir"])]
ds_files = [e for e in ds_files if '.csv' in e]
if p_train['dsVal']:
    dfVal = pd.read_csv(p_train['dsVal'])
    print(f'VALDIACION : {len(dfVal)}')
    print('--------------------------------------------------')
print(f'Cantidad de datasets : {len(ds_files)}')  
tempDF = pd.read_csv(ds_files[0])
print(f'TRAIN: {len(tempDF)}')
tempDF.head(2)

VALDIACION : 3207
--------------------------------------------------
Cantidad de datasets : 2
TRAIN: 6008


,Unnamed: 0.1,Unnamed: 0,nombre,codigo,XO,XA,longitud,latitud,altura,dato,...,99%,75%,umb1,umb2,fecha,flag,flagV2,imagen,clase,DA
0,0,37890,PICHARI,47E880E2,539,772,-73.83952,-12.52219,570.0,0.1,...,6.319,0.0,4.2,24.0,2021-11-06-00,M0000002,C01,539--772--2021-11-06-00,0,0
1,1,37891,SAN PABLO,4729658E,607,799,-72.61992,-13.02506,1237.0,0.2,...,1.600,0.0,2.6,6.9,2021-12-14-04,M0000002,C01,607--799--2021-12-14-04,0,0


In [22]:
%%time
resultados = trainModel(p_train,path_imagenes,path_base,products,times)

Cantidad de datasets encontrados : 2
Datasets a usar : 2
DIRECTORIO BASE : C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/NewTesis/Archivos/Resultados\Clasificacion\Clasificacion_RNN_UMB1-DATO-ALT-2CONV-20220612_00
__________________________________________________
DATASET : 1/2
__________________________________________________
Inicio de la prueba N°: 1/1
Dataset: C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/NewTesis/Archivos/Dataset/Clasificacion/Entrenamiento/SplitConDA_20C_DM_V2/CLASE_TrainDS_0
- Batch size:  32
__________________________________________________
Tamaño del dataset: Train 6008  - Val 3207
Epoch 1/40
188/188 [==============================] - ETA: 0s - loss: 1.1103 - acc: 0.5932 - true_positives: 1819.0000 - true_negatives: 1745.0000 - false_positives: 1259.0000 - false_negatives: 1185.0000
Epoch 1: saving model to C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/NewTesis/Archivos/Resultados\Clasificacion\Clasificacion_RNN_UMB1-DATO-ALT-2CONV-20220612_00\CLASE_TrainDS_0\Model_01_Cl

Epoch 11/40
188/188 [==============================] - ETA: 0s - loss: 0.5634 - acc: 0.7240 - true_positives: 1966.0000 - true_negatives: 2384.0000 - false_positives: 620.0000 - false_negatives: 1038.0000
Epoch 11: saving model to C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/NewTesis/Archivos/Resultados\Clasificacion\Clasificacion_RNN_UMB1-DATO-ALT-2CONV-20220612_00\CLASE_TrainDS_0\Model_11_Clasificacion_clase_20220612_004451.hdf5
188/188 [==============================] - 335s 2s/step - loss: 0.5634 - acc: 0.7240 - true_positives: 1966.0000 - true_negatives: 2384.0000 - false_positives: 620.0000 - false_negatives: 1038.0000 - val_loss: 0.3403 - val_acc: 0.7677 - val_true_positives: 2434.0000 - val_true_negatives: 28.0000 - val_false_positives: 22.0000 - val_false_negatives: 723.0000
Epoch 12/40
188/188 [==============================] - ETA: 0s - loss: 0.5676 - acc: 0.7247 - true_positives: 1989.0000 - true_negatives: 2365.0000 - false_positives: 639.0000 - false_negatives: 1015.0000
Epo

188/188 [==============================] - 335s 2s/step - loss: 0.5371 - acc: 0.7533 - true_positives: 2228.0000 - true_negatives: 2298.0000 - false_positives: 706.0000 - false_negatives: 776.0000 - val_loss: 0.4233 - val_acc: 0.7746 - val_true_positives: 2451.0000 - val_true_negatives: 33.0000 - val_false_positives: 17.0000 - val_false_negatives: 706.0000
Epoch 22/40
188/188 [==============================] - ETA: 0s - loss: 0.5520 - acc: 0.7473 - true_positives: 2179.0000 - true_negatives: 2311.0000 - false_positives: 693.0000 - false_negatives: 825.0000
Epoch 22: saving model to C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/NewTesis/Archivos/Resultados\Clasificacion\Clasificacion_RNN_UMB1-DATO-ALT-2CONV-20220612_00\CLASE_TrainDS_0\Model_22_Clasificacion_clase_20220612_004451.hdf5
188/188 [==============================] - 335s 2s/step - loss: 0.5520 - acc: 0.7473 - true_positives: 2179.0000 - true_negatives: 2311.0000 - false_positives: 693.0000 - false_negatives: 825.0000 - val_loss: 0

188/188 [==============================] - 343s 2s/step - loss: 0.7214 - acc: 0.6396 - true_positives_1: 1673.0000 - true_negatives_1: 2170.0000 - false_positives_1: 834.0000 - false_negatives_1: 1331.0000 - val_loss: 0.4587 - val_acc: 0.6985 - val_true_positives_1: 2204.0000 - val_true_negatives_1: 36.0000 - val_false_positives_1: 14.0000 - val_false_negatives_1: 953.0000
Epoch 3/40
188/188 [==============================] - ETA: 0s - loss: 0.6635 - acc: 0.6588 - true_positives_1: 1723.0000 - true_negatives_1: 2235.0000 - false_positives_1: 769.0000 - false_negatives_1: 1281.0000
Epoch 3: saving model to C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/NewTesis/Archivos/Resultados\Clasificacion\Clasificacion_RNN_UMB1-DATO-ALT-2CONV-20220612_00\CLASE_TrainDS_1\Model_03_Clasificacion_clase_20220612_035946.hdf5
188/188 [==============================] - 473s 3s/step - loss: 0.6635 - acc: 0.6588 - true_positives_1: 1723.0000 - true_negatives_1: 2235.0000 - false_positives_1: 769.0000 - false_neg

Epoch 13/40
188/188 [==============================] - ETA: 0s - loss: 0.5552 - acc: 0.7395 - true_positives_1: 2090.0000 - true_negatives_1: 2353.0000 - false_positives_1: 651.0000 - false_negatives_1: 914.0000
Epoch 13: saving model to C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/NewTesis/Archivos/Resultados\Clasificacion\Clasificacion_RNN_UMB1-DATO-ALT-2CONV-20220612_00\CLASE_TrainDS_1\Model_13_Clasificacion_clase_20220612_035946.hdf5
188/188 [==============================] - 477s 3s/step - loss: 0.5552 - acc: 0.7395 - true_positives_1: 2090.0000 - true_negatives_1: 2353.0000 - false_positives_1: 651.0000 - false_negatives_1: 914.0000 - val_loss: 0.5679 - val_acc: 0.6701 - val_true_positives_1: 2112.0000 - val_true_negatives_1: 37.0000 - val_false_positives_1: 13.0000 - val_false_negatives_1: 1045.0000
Epoch 14/40
188/188 [==============================] - ETA: 0s - loss: 0.5258 - acc: 0.7578 - true_positives_1: 2192.0000 - true_negatives_1: 2361.0000 - false_positives_1: 643.0000 - f

In [23]:
%%time
"""
# Definimos las varibles para las iteraciones
Los parametros que van a cambiar son:
- Canales (products)
- Tiempos (Minutos de las imagenes)
- margen
"""

modelTipo = 'Clasificacion'
idProject = datetime.today().strftime("%Y%m%d_%H")
p_train = {
            # Reportes
          'Proyect'  : f'{modelTipo}_RNN_UMB2-DATO-ALT-2CONV-{idProject}', # TesisDiego
          'record'   : True,  # Grabar los resultados en  excels    
          'WANDB'    : False, # Grabar los resultados en WANDB
    
            # Datos del modelo
          'redTipo'  : modelTipo, # Clasificacion / Regresion
          'rnn'      : True,  # Redes recurrentes
          'inputs'   : ['imagen','umb1','dato','altura'],
          'outputs'  : 'clase',       # clase / dato
          'num_class': 2,             # Solo se usa en el modelo de clasificacion
          'meanMatrizImagen' : False, # !!! RNN modelos SIEMPRE EN FALSE  !!!
          
        
            # Variables del entrenamiento          
          'lr'       : 0.001,
          'batch'    : 32,        
          'val_split': 1,     # ¡¡ Si dsVal existe, este valor se ignora !!
          'epocas'   : 40,  
          'paciencia': 10,        # 0 = No paciencia  
    
    
           # Dataset
          'dsDir'    : f'{path_base}/Archivos/Dataset/{modelTipo}/Entrenamiento/SplitConDA_20C_DM_V2/',  #SplitConDA_DM_DFAD_DUD
          'dsVal'    : f'{path_base}/Archivos/Dataset/{modelTipo}/Validacion/ClaseV2_DUD_ValidacionDS.csv',
          'maxDS'    : -1,      #-1 = Todos
          'dataset'  : 1,     # 1 = 100% del ds
          'DA'       : True,  # Usaulmente para clasificacion
                    
           # Hiper parametros 
          'canales'  : [3],
          'tiempos'  : [6],
          'margen'   : [30],
          'runs'     : 1
         }


resultados = trainModel(p_train,path_imagenes,path_base,products,times)

Cantidad de datasets encontrados : 2
Datasets a usar : 2
DIRECTORIO BASE : C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/NewTesis/Archivos/Resultados\Clasificacion\Clasificacion_RNN_UMB2-DATO-ALT-2CONV-20220612_06
__________________________________________________
DATASET : 1/2
__________________________________________________
Inicio de la prueba N°: 1/1
Dataset: C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/NewTesis/Archivos/Dataset/Clasificacion/Entrenamiento/SplitConDA_20C_DM_V2/CLASE_TrainDS_0
- Batch size:  32
__________________________________________________
Tamaño del dataset: Train 6008  - Val 3207
Epoch 1/40
188/188 [==============================] - ETA: 0s - loss: 0.8202 - acc: 0.5847 - true_positives_2: 1999.0000 - true_negatives_2: 1514.0000 - false_positives_2: 1490.0000 - false_negatives_2: 1005.0000
Epoch 1: saving model to C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/NewTesis/Archivos/Resultados\Clasificacion\Clasificacion_RNN_UMB2-DATO-ALT-2CONV-20220612_06\CLASE_TrainDS_0\Mod

188/188 [==============================] - 336s 2s/step - loss: 0.5427 - acc: 0.7572 - true_positives_2: 2318.0000 - true_negatives_2: 2231.0000 - false_positives_2: 773.0000 - false_negatives_2: 686.0000 - val_loss: 0.4929 - val_acc: 0.7574 - val_true_positives_2: 2396.0000 - val_true_negatives_2: 33.0000 - val_false_positives_2: 17.0000 - val_false_negatives_2: 761.0000
Epoch 11/40
188/188 [==============================] - ETA: 0s - loss: 0.5356 - acc: 0.7562 - true_positives_2: 2304.0000 - true_negatives_2: 2239.0000 - false_positives_2: 765.0000 - false_negatives_2: 700.0000
Epoch 11: saving model to C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/NewTesis/Archivos/Resultados\Clasificacion\Clasificacion_RNN_UMB2-DATO-ALT-2CONV-20220612_06\CLASE_TrainDS_0\Model_11_Clasificacion_clase_20220612_061039.hdf5
188/188 [==============================] - 337s 2s/step - loss: 0.5356 - acc: 0.7562 - true_positives_2: 2304.0000 - true_negatives_2: 2239.0000 - false_positives_2: 765.0000 - false_neg

Epoch 21/40
188/188 [==============================] - ETA: 0s - loss: 0.5247 - acc: 0.7595 - true_positives_2: 2237.0000 - true_negatives_2: 2326.0000 - false_positives_2: 678.0000 - false_negatives_2: 767.0000
Epoch 21: saving model to C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/NewTesis/Archivos/Resultados\Clasificacion\Clasificacion_RNN_UMB2-DATO-ALT-2CONV-20220612_06\CLASE_TrainDS_0\Model_21_Clasificacion_clase_20220612_061039.hdf5
188/188 [==============================] - 457s 2s/step - loss: 0.5247 - acc: 0.7595 - true_positives_2: 2237.0000 - true_negatives_2: 2326.0000 - false_positives_2: 678.0000 - false_negatives_2: 767.0000 - val_loss: 0.5710 - val_acc: 0.6925 - val_true_positives_2: 2187.0000 - val_true_negatives_2: 34.0000 - val_false_positives_2: 16.0000 - val_false_negatives_2: 970.0000
__________________________________________________
DATASET : 2/2
__________________________________________________
Inicio de la prueba N°: 1/1
Dataset: C:/Users/Shounen/Desktop/Ciclo XI

Epoch 10/40
188/188 [==============================] - ETA: 0s - loss: 0.5455 - acc: 0.7530 - true_positives_3: 2155.0000 - true_negatives_3: 2369.0000 - false_positives_3: 635.0000 - false_negatives_3: 849.0000
Epoch 10: saving model to C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/NewTesis/Archivos/Resultados\Clasificacion\Clasificacion_RNN_UMB2-DATO-ALT-2CONV-20220612_06\CLASE_TrainDS_1\Model_10_Clasificacion_clase_20220612_081859.hdf5
188/188 [==============================] - 414s 2s/step - loss: 0.5455 - acc: 0.7530 - true_positives_3: 2155.0000 - true_negatives_3: 2369.0000 - false_positives_3: 635.0000 - false_negatives_3: 849.0000 - val_loss: 0.5068 - val_acc: 0.7265 - val_true_positives_3: 2297.0000 - val_true_negatives_3: 33.0000 - val_false_positives_3: 17.0000 - val_false_negatives_3: 860.0000
Epoch 11/40
188/188 [==============================] - ETA: 0s - loss: 0.5346 - acc: 0.7585 - true_positives_3: 2176.0000 - true_negatives_3: 2381.0000 - false_positives_3: 623.0000 - fa